In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from math import sqrt
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.discrete_allocation import get_latest_prices
from dateutil.relativedelta import relativedelta
import copy
import warnings
warnings.filterwarnings('ignore')

tickers = ['AAPL', 'GOOG', 'MSFT', 'AMZN', 'INTC', 'AMD', 'NVDA', 'F', 'TSLA', 'JPM', 'MS', 'VOO']
data_for_portfolio_5y = yf.download(tickers, start='2016-01-01', end='2021-01-01')['Close'].dropna()
data_for_portfolio_5y.tail()

[*********************100%***********************]  12 of 12 completed


,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,TSLA,VOO
Date,,,,,,,,,,,,
2020-12-24,131.970001,91.809998,158.634506,8.86,86.942497,47.070000,124.519997,68.089996,222.750000,129.937500,220.589996,339.160004
2020-12-28,136.690002,91.599998,164.197998,8.89,88.804497,47.070000,125.339996,68.050003,224.960007,129.000000,221.229996,342.109985
2020-12-29,134.869995,90.620003,166.100006,8.82,87.935997,49.389999,125.010002,67.709999,224.149994,129.432495,221.996674,341.390015
2020-12-30,133.720001,92.290001,164.292496,8.86,86.975998,48.750000,125.360001,67.839996,221.679993,131.457504,231.593338,341.850006
2020-12-31,132.690002,91.709999,162.846497,8.79,87.594002,49.820000,127.070000,68.529999,222.419998,130.550003,235.223328,343.690002


In [2]:
# Create initial portfolio
mu = mean_historical_return(data_for_portfolio_5y)
S = CovarianceShrinkage(data_for_portfolio_5y).ledoit_wolf()
ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
weights = ef.max_sharpe()
initial_weights = ef.clean_weights()
print(initial_weights, '\n')

# Initial buy stocks
balance_usd = 100_000
balance_remain = balance_usd
stocks_initial = {}
latest_prices_data_dec_2020 = get_latest_prices(data_for_portfolio_5y)
for ticker in initial_weights:
    to_spend_usd = balance_usd * initial_weights[ticker]
    one_stock_price = latest_prices_data_dec_2020[ticker]
    stocks_number = to_spend_usd // one_stock_price
    stocks_initial[ticker] = stocks_number
    balance_remain = balance_remain - (stocks_number * one_stock_price)
print('Portfolio:', stocks_initial)
print('Remaining balance: USD', balance_remain)

OrderedDict([('AAPL', 0.1135), ('AMD', 0.27372), ('AMZN', 0.11721), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.0), ('NVDA', 0.28702), ('TSLA', 0.20854), ('VOO', 0.0)]) 

Portfolio: {'AAPL': 85.0, 'AMD': 298.0, 'AMZN': 71.0, 'F': 0.0, 'GOOG': 0.0, 'INTC': 0.0, 'JPM': 0.0, 'MS': 0.0, 'MSFT': 0.0, 'NVDA': 219.0, 'TSLA': 88.0, 'VOO': 0.0}
Remaining balance: USD 539.5653076171875


In [3]:
# Calculate MACD, RSI, and S&R signals.
data_2021 = yf.download(tickers, start='2020-12-12', end='2022-01-01')['Close'].dropna()

short_ma = 5                                       # 5 минулих днів для "короткого" МА 
long_ma = 12                                       # 12 минулих днів для "довгого" МА 
rsi_period = 14                                    # RSI рахується за 14 минулих днів 
rsi_oversold = 30                                  # Параметри для індикатора RSI 
rsi_overbought = 70 
sr_sell = 0.7                                      # Параметри для індикатора S&R
sr_buy = 0.3 
start = max(long_ma, rsi_period)

for ticker in data_2021.columns:
    data_2021['MA_' + ticker + str(short_ma)] = data_2021[ticker].rolling(short_ma).mean()
    data_2021['MA_' + ticker + str(long_ma)] = data_2021[ticker].rolling(long_ma).mean() 
    data_2021['return_' + ticker] = data_2021[ticker].pct_change() 
    data_2021['Up_' + ticker] = np.maximum(data_2021[ticker].diff(),0) 
    data_2021['Down_' + ticker] = np.maximum(-data_2021[ticker].diff(),0) 
    data_2021['RS_' + ticker] = data_2021['Up_' + ticker].rolling(rsi_period).mean()/data_2021['Down_' + ticker].rolling(rsi_period).mean() 
    data_2021['RSI_' + ticker] = 100 - 100/(1 + data_2021['RS_' + ticker]) 
    data_2021['S&R_' + ticker] = (data_2021[ticker]/(10**np.floor(np.log10(data_2021[ticker]))))%1 
    data_2021['MACD_signal_' + ticker] = 2*(data_2021['MA_' + ticker + str(short_ma)] > data_2021['MA_' + ticker + str(long_ma)]) -1 
    data_2021['RSI_signal_' + ticker] = 1*(data_2021['RSI_' + ticker] < rsi_oversold) -1*(data_2021['RSI_' + ticker] > rsi_overbought) 
    data_2021['S&R_signal_' + ticker] = 1*(data_2021['S&R_' + ticker] < sr_buy) -1*(data_2021['S&R_' + ticker] > sr_sell) 

data = data_2021.drop(data_2021.index[range(14)])
#print(data.to_string())
data.head(3)

[*********************100%***********************]  12 of 12 completed


,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,...,MA_VOO12,return_VOO,Up_VOO,Down_VOO,RS_VOO,RSI_VOO,S&R_VOO,MACD_signal_VOO,RSI_signal_VOO,S&R_signal_VOO
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,129.410004,92.300003,159.331497,8.52,86.412003,49.669998,125.870003,68.330002,217.690002,131.134995,...,340.415003,-0.013559,0.000000,4.660004,1.217737,54.908985,0.3903,1,0,0
2021-01-05,131.009995,92.769997,160.925507,8.65,87.045998,50.610001,125.650002,69.489998,217.899994,134.047501,...,340.506671,0.006578,2.230011,0.000000,1.634519,62.042412,0.4126,1,0,0
2021-01-06,126.599998,90.330002,156.919006,8.84,86.764503,51.099998,131.550003,73.680000,212.250000,126.144997,...,340.616669,0.006066,2.069977,0.000000,1.374617,57.887952,0.4333,1,0,0


In [4]:
data.tail(3)

,AAPL,AMD,AMZN,F,GOOG,INTC,JPM,MS,MSFT,NVDA,...,MA_VOO12,return_VOO,Up_VOO,Down_VOO,RS_VOO,RSI_VOO,S&R_VOO,MACD_signal_VOO,RSI_signal_VOO,S&R_signal_VOO
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-29,179.380005,148.259995,169.201004,20.559999,146.504501,51.830002,158.559998,98.730003,341.950012,300.010010,...,430.458336,0.001209,0.529999,0.000000,1.318102,56.861268,0.3901,1,0,0
2021-12-30,178.199997,145.149994,168.644501,20.469999,146.002502,51.740002,158.479996,98.800003,339.320007,295.859985,...,431.188334,-0.002825,0.000000,1.240021,1.423401,58.735673,0.3777,1,0,0
2021-12-31,177.570007,143.899994,166.716995,20.770000,144.679504,51.500000,158.350006,98.160004,336.320007,294.109985,...,432.061668,-0.002741,0.000000,1.199982,1.165480,53.820870,0.3657,1,0,0


In [5]:
def check_tickers_for_sell(tickers_sell, now, current_weights):
    ticker_percent_sell = {}
    end_date = now.strftime('%Y-%m-%d')
    start_date = (now - relativedelta(years=5)).strftime('%Y-%m-%d')
    # Get data for all tickers for last 5 years
    data_for_tickers = yf.download(tickers, start_date, end_date, progress=False)['Close'].dropna()
    
    # Generate portfolio on this data
    mu = mean_historical_return(data_for_tickers)
    S = CovarianceShrinkage(data_for_tickers).ledoit_wolf()
    ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
    weights = ef.max_sharpe()
    new_weights = ef.clean_weights()
    
    for ticker in tickers_sell:
        if new_weights[ticker] == 0:
            ticker_percent_sell[ticker] = 100
        elif new_weights[ticker] < current_weights[ticker]:
            delta = current_weights[ticker] - new_weights[ticker]
            sell_percent = (delta * 100) // current_weights[ticker]
            ticker_percent_sell[ticker] = sell_percent
    return ticker_percent_sell

In [6]:
def select_ticker_for_sell(data, date, filtered_sell_tickers_with_percents, stocks_current):
    sell_ticker = ''
    sell_profit = 0
    stocks_to_sell = 0
    for ticker in filtered_sell_tickers_with_percents:
        no_of_stocks = stocks_current[ticker]
        #print(ticker, 'no_of_stocks', no_of_stocks)
        cur_price = data[ticker][date]
        #print(ticker, 'cur_price', cur_price)
        sell_percent = (filtered_sell_tickers_with_percents[ticker] / 100)
        #print(ticker, 'sell_percent', sell_percent)
        potentail_profit = no_of_stocks * cur_price * sell_percent
        #print(ticker, 'potentail_profit', potentail_profit)
        if potentail_profit > sell_profit:
            sell_profit = potentail_profit
            sell_ticker = ticker
            stocks_to_sell = no_of_stocks * sell_percent
    return [sell_ticker, sell_profit, stocks_to_sell]

In [7]:
def select_ticker_for_buy(tickers_buy, now):
    if len(tickers_buy) == 1:
        return tickers_buy[0]
    end_date = now.strftime('%Y-%m-%d')
    start_date = (now - relativedelta(years=5)).strftime('%Y-%m-%d')
    # Get data for all tickers for last 5 years period
    data_for_tickers_last_year = yf.download(tickers_buy, start_date, end_date, progress=False)['Close'].dropna()
    ticker_percent_delta = {}
    for t in tickers_buy:
        first_price = data_for_tickers_last_year[t][0]
        last_price = data_for_tickers_last_year[t][len(data_for_tickers_last_year[t]) - 1]
        delta = last_price - first_price
        delta_abs = abs(delta)
        percent = (delta_abs * 100) / first_price
        if delta > 0:
            ticker_percent_delta[t] = percent
        else:
            ticker_percent_delta[t] = -1 * percent
    sorted_by_delta = dict(sorted(ticker_percent_delta.items(), key=lambda x:x[1], reverse=True))
    return list(sorted_by_delta.keys())[0]

In [8]:
print('Start weights:', initial_weights)
clean_result = balance_remain
for stock in initial_weights:
    clean_result = clean_result + (stocks_initial[stock] * data[stock][len(data[stock]) - 1])
print('Without any changes balance on the end of 2021 would be', clean_result)
print('------------------------------------')

Start weights: OrderedDict([('AAPL', 0.1135), ('AMD', 0.27372), ('AMZN', 0.11721), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.0), ('NVDA', 0.28702), ('TSLA', 0.20854), ('VOO', 0.0)])
Without any changes balance on the end of 2021 would be 165761.08842468262
------------------------------------


In [9]:
strategy = 'MACD_signal_'                 # MACD_signal_        or        RSI_signal_        or        S&R_signal_
strategy_weights = copy.deepcopy(initial_weights)
stocks_strategy = copy.deepcopy(stocks_initial)
balance_strategy = balance_remain

balance_buy_and_hold_dict = {}
balance_strategy_dict = {}

for d in data.index: 
    if d.weekday() == 4:                   # every Friday
        tickers_sell = []
        tickers_buy = []
        for t in strategy_weights:
            if data[strategy + t][d] == -1:
                tickers_sell.append(t)
            if data[strategy + t][d] == 1:
                tickers_buy.append(t)
        print(d, 'Sell candidates:', tickers_sell, 'Buy candidates:', tickers_buy)
        if len(tickers_sell) > 0 and len(tickers_buy) > 0:
            filtered_sell_tickers_with_percents = check_tickers_for_sell(tickers_sell, d, strategy_weights)
            print('Filtered sell candidates:', filtered_sell_tickers_with_percents)
            if len(filtered_sell_tickers_with_percents) > 0:
                ticker_for_sell_data = select_ticker_for_sell(data, d, filtered_sell_tickers_with_percents, stocks_strategy)
                will_receive_usd = ticker_for_sell_data[1]
                # If this ticker is present in current portfolio
                if will_receive_usd > 0:
                    ticker_for_sell = ticker_for_sell_data[0]
                    stocks_to_sell = ticker_for_sell_data[2]
                    sell_percents = filtered_sell_tickers_with_percents[ticker_for_sell]
                    ticker_for_buy = select_ticker_for_buy(tickers_buy, d)
                    # If we can buy other ticker on that money
                    if (will_receive_usd + balance_strategy) > data[ticker_for_buy][d]:
                        print('Will sell',ticker_for_sell,'and get',will_receive_usd,'$ after selling (',sell_percents,'% )')
                        print('Will buy', ticker_for_buy)
                        # Sell percent of ticker_for_sell and add that money to balance
                        stocks_strategy[ticker_for_sell] = stocks_strategy[ticker_for_sell] - stocks_to_sell
                        balance_strategy = balance_strategy + will_receive_usd
                        # Buy ticker_for_buy on all balance
                        no_of_buy_stocks = balance_strategy // data[ticker_for_buy][d]
                        stocks_strategy[ticker_for_buy] = stocks_strategy[ticker_for_buy] + no_of_buy_stocks
                        balance_strategy = balance_strategy - (data[ticker_for_buy][d] * no_of_buy_stocks)
                        # Recalculate new weights for all portfolio
                        # calculate all prices for now + balance - потом высчитать сколько в процентах сейчас каждый ФИ
                        cur_balance = balance_strategy
                        stock_usd = {}
                        for stock in stocks_strategy:
                            stock_usd[stock] = stocks_strategy[stock] * data[stock][d]
                            cur_balance = cur_balance + stock_usd[stock]
                        for stock in stocks_strategy:
                            strategy_weights[stock] = stock_usd[stock] / cur_balance
                        print('Current weights are', strategy_weights)
                        print('Current balance =', cur_balance)
                    else:
                        print("No changes because sell profit is not enough for buying at least one stock.")
                else:
                    print("No changes because all sell candidates are not present in current portfolio.")
            else:
                print("No changes because all sell candidates failed check.")
        else:
            print("No changes because no sell candidates or no buy candidates.")
        print('------------------------------------')
    # Calculate current balance for stocks_initial
    today_balance_buy_and_hold = balance_remain
    for stock in stocks_initial:
        today_balance_buy_and_hold = today_balance_buy_and_hold + (data[stock][d] * stocks_initial[stock])
    balance_buy_and_hold_dict[d] = today_balance_buy_and_hold
    # Calculate current balance for stocks_strategy
    today_balance_strategy = balance_strategy
    for stock in stocks_strategy:
        today_balance_strategy = today_balance_strategy + (data[stock][d] * stocks_strategy[stock])
    balance_strategy_dict[d] = today_balance_strategy

print('\nWith strategy final weights are:', strategy_weights)        
clean_result = balance_strategy
for stock in strategy_weights:
    clean_result = clean_result + (stocks_strategy[stock] * data[stock][len(data[stock]) - 1])
print('\nWith strategy balance on the end of 2021 would be', clean_result)

2021-01-08 00:00:00 Sell candidates: ['AAPL', 'AMZN', 'F', 'MSFT'] Buy candidates: ['AMD', 'GOOG', 'INTC', 'JPM', 'MS', 'NVDA', 'TSLA', 'VOO']
Filtered sell candidates: {'AAPL': 19.0, 'AMZN': 2.0, 'F': 100, 'MSFT': 100}
Will sell AAPL and get 2132.6075492858886 $ after selling ( 19.0 % )
Will buy AMD
Current weights are OrderedDict([('AAPL', 0.08565930701908782), ('AMD', 0.29050199192915693), ('AMZN', 0.10645259155545839), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.0), ('NVDA', 0.27394797524969966), ('TSLA', 0.24321264501313763), ('VOO', 0.0)])
Current balance = 106137.2433013916
------------------------------------
2021-01-15 00:00:00 Sell candidates: ['AAPL', 'AMD', 'AMZN', 'GOOG', 'MSFT'] Buy candidates: ['F', 'INTC', 'JPM', 'MS', 'NVDA', 'TSLA', 'VOO']
Filtered sell candidates: {'AAPL': 77.0, 'GOOG': 100, 'MSFT': 100}
Will sell AAPL and get 6740.263497642516 $ after selling ( 77.0 % )
Will buy TSLA
Current weights are OrderedDict([('AAPL', 0.019

Filtered sell candidates: {'AMD': 24.0, 'F': 100, 'INTC': 100, 'JPM': 100, 'MS': 100}
Will sell AMD and get 6140.411265463461 $ after selling ( 24.0 % )
Will buy TSLA
Current weights are OrderedDict([('AAPL', 0.0), ('AMD', 0.1921376691831436), ('AMZN', 0.13700035656161128), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.14708811533524513), ('NVDA', 0.2323386009439405), ('TSLA', 0.2901217661346814), ('VOO', 0.0)])
Current balance = 101201.57987048969
------------------------------------
2021-04-30 00:00:00 Sell candidates: ['AAPL', 'INTC', 'MSFT', 'NVDA', 'TSLA'] Buy candidates: ['AMD', 'AMZN', 'F', 'GOOG', 'JPM', 'MS', 'VOO']
Filtered sell candidates: {'INTC': 100, 'MSFT': 69.0, 'TSLA': 47.0}
Will sell TSLA and get 13421.942413507079 $ after selling ( 47.0 % )
Will buy AMD
Current weights are OrderedDict([('AAPL', 0.0), ('AMD', 0.3280766510808079), ('AMZN', 0.1442582919952446), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT'

Filtered sell candidates: {'AMD': 27.0, 'AMZN': 100, 'F': 100, 'GOOG': 100, 'INTC': 100, 'JPM': 100, 'MS': 100, 'NVDA': 13.0, 'TSLA': 29.0, 'VOO': 100}
Will sell TSLA and get 9140.610711530639 $ after selling ( 29.0 % )
Will buy AAPL
Current weights are OrderedDict([('AAPL', 0.2513548409410537), ('AMD', 0.17443040612790878), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.1501324899863273), ('NVDA', 0.23029918873125824), ('TSLA', 0.19366352897221414), ('VOO', 0.0)])
Current balance = 115554.72880399844
------------------------------------
2021-08-27 00:00:00 Sell candidates: ['F'] Buy candidates: ['AAPL', 'AMD', 'AMZN', 'GOOG', 'INTC', 'JPM', 'MS', 'MSFT', 'NVDA', 'TSLA', 'VOO']
Filtered sell candidates: {'F': 100}
No changes because all sell candidates are not present in current portfolio.
------------------------------------
2021-09-03 00:00:00 Sell candidates: [] Buy candidates: ['AAPL', 'AMD', 'AMZN', 'F', 'GOOG', 'INTC', 'JPM', 'MS', 

Filtered sell candidates: {'AMZN': 100}
No changes because all sell candidates are not present in current portfolio.
------------------------------------

With strategy final weights are: OrderedDict([('AAPL', 0.2674967671432427), ('AMD', 0.20111227765345654), ('AMZN', 0.0), ('F', 0.0), ('GOOG', 0.0), ('INTC', 0.0), ('JPM', 0.0), ('MS', 0.0), ('MSFT', 0.12614918078889836), ('NVDA', 0.08865572613282663), ('TSLA', 0.3154258845901279), ('VOO', 0.0)])

With strategy balance on the end of 2021 would be 156703.31707752252


In [10]:
high_buy_and_hold = 0
low_buy_and_hold = 1_000_000_000
high_strategy = 0
low_strategy = 1_000_000_000
biggest_delta = 0
biggest_delta_day = ''
final_delta_usd = 0
final_delta_percent = 0

for d in balance_buy_and_hold_dict:
    if balance_buy_and_hold_dict[d] > high_buy_and_hold:
        high_buy_and_hold = balance_buy_and_hold_dict[d]
    if balance_buy_and_hold_dict[d] < low_buy_and_hold:
        low_buy_and_hold = balance_buy_and_hold_dict[d]
    if balance_strategy_dict[d] > high_strategy:
        high_strategy = balance_strategy_dict[d]
    if balance_strategy_dict[d] < low_strategy:
        low_strategy = balance_strategy_dict[d]
    delta = balance_strategy_dict[d] - balance_buy_and_hold_dict[d]
    delta_abs = abs(delta)
    percent = delta_abs / balance_buy_and_hold_dict[d]
    percent_change = percent
    if delta < 0:
        percent_change = -1 * percent_change
    if percent > abs(biggest_delta):
        biggest_delta = percent_change
        biggest_delta_day = d
    final_delta_usd = balance_strategy_dict[d] - balance_buy_and_hold_dict[d]
    final_delta_percent = percent_change
    print(d.strftime('%Y-%m-%d'), '\tBuy and hold balance:\t', "{:.2f}".format(balance_buy_and_hold_dict[d]), 
          '\tStrategy balance:\t', "{:.2f}".format(balance_strategy_dict[d]), 
          '\tDelta:\t', "{:.6f}".format(percent_change))

print('\nhigh_buy_and_hold', high_buy_and_hold)
print('low_buy_and_hold', low_buy_and_hold)
print('\nhigh_strategy', high_strategy)
print('low_strategy', low_strategy)
print('\nbiggest_delta', biggest_delta)
print('biggest_delta_day', biggest_delta_day)
print('\nfinal_delta_usd', final_delta_usd)
print('final_delta_percent', final_delta_percent)

2021-01-04 	Buy and hold balance:	 100482.50 	Strategy balance:	 100482.50 	Delta:	 0.000000
2021-01-05 	Buy and hold balance:	 101666.21 	Strategy balance:	 101666.21 	Delta:	 0.000000
2021-01-06 	Buy and hold balance:	 99161.32 	Strategy balance:	 99161.32 	Delta:	 0.000000
2021-01-07 	Buy and hold balance:	 104411.65 	Strategy balance:	 104411.65 	Delta:	 0.000000
2021-01-08 	Buy and hold balance:	 106137.24 	Strategy balance:	 106137.24 	Delta:	 0.000000
2021-01-11 	Buy and hold balance:	 105164.80 	Strategy balance:	 105289.14 	Delta:	 0.001182
2021-01-12 	Buy and hold balance:	 105432.30 	Strategy balance:	 105506.63 	Delta:	 0.000705
2021-01-13 	Buy and hold balance:	 104951.79 	Strategy balance:	 104892.13 	Delta:	 -0.000569
2021-01-14 	Buy and hold balance:	 103350.07 	Strategy balance:	 103294.66 	Delta:	 -0.000536
2021-01-15 	Buy and hold balance:	 101049.46 	Strategy balance:	 100950.40 	Delta:	 -0.000980
2021-01-19 	Buy and hold balance:	 102438.68 	Strategy balance:	 1024